In [1]:
import numpy as np
import pandas as pd
import datetime as dt
from collections import Counter
import re
import warnings # Turn off warnings
warnings.filterwarnings('ignore')
# Pandas options
pd.set_option("max_rows", 30)
pd.set_option("max_columns", None)
pd.set_option("precision", 3)

In [2]:
def format_turnstile_df(turnstile_csv_df):
    """
    turnstile_csv_df is the dataframe of the imported turnstile.csv file
    Warning: Run once per file ONLY. The function will throw an error if run twice on the same file.
    """
    global turnstile_df
    turnstile_df = turnstile_csv_df
    
    # Reformatting steps (Warning: May take a few seconds.)
    turnstile_df.columns = turnstile_df.columns.str.replace(' ','') # Remove spaces in column names
    turnstile_df.columns = turnstile_df.columns.str.replace('/','') # Remove "/" in column names
    turnstile_df['DATETIMERAW'] = "" # Create new column called 'DATETIMERAW'
    turnstile_df['DATETIMERAW'] = turnstile_df.DATE + turnstile_df.TIME # Populate 'DATETIMERAW' with date and time concatenated string
    turnstile_df.DATETIMERAW = turnstile_df.DATETIMERAW.apply(lambda x : dt.datetime.strptime(x, "%m/%d/%Y%H:%M:%S")) # Convert DATETIMERAW into datetime object
    turnstile_df.TIME = turnstile_df.TIME.apply(lambda x : dt.datetime.strptime(x, "%H:%M:%S")) # Convert TIME into datetime object
    turnstile_df = turnstile_df.drop(columns = ['DATE']) # Drop DATE column

    
    
def valuecount(timestamp1, timestamp2):
    """
    This function references turnstile_df, which is the output of format_turnstile_df().
    It grabs the data at two individual time stamps and outputs the difference in the count between those timestamps.
    Both timestamp1 and timestamp2 should be datetime objects. Also, timestamp2 should be greater than timestamp1.
    This function is not run as a standalone function. It is run within the grabweek() function.
    """
    global value_count_df
    
    # Grab data
    turnstile_df1 = turnstile_df.loc[turnstile_df['DATETIMERAW'].isin([timestamp1])].reset_index(drop=True) # Grab dataset for timestamp1
    turnstile_df2 = turnstile_df.loc[turnstile_df['DATETIMERAW'].isin([timestamp2])].reset_index(drop=True) # Grab dataset for timestamp2
    
    # Merge two datasets, drop any rows that don't form a complete dataset
    turnstile_df_merge12 = turnstile_df1.merge(turnstile_df2.drop_duplicates(), on=["CA", "UNIT", "SCP", "STATION", "LINENAME", "DIVISION", "DESC"], how='outer').dropna().reset_index(drop=True)
    
    # Create a new dataframe with essential data
    turnstile_df_valuecounts = turnstile_df_merge12[["STATION"]] # Grabbing "STATION" name data
    turnstile_df_valuecounts["ENTRIES DIFFERENCE"] = (turnstile_df_merge12['ENTRIES_x'] - turnstile_df_merge12['ENTRIES_y']).abs() # Grabbing "ENTRIES" difference counts
    turnstile_df_valuecounts["EXITS DIFFERENCE"] = (turnstile_df_merge12['EXITS_x'] - turnstile_df_merge12['EXITS_y']).abs() # Grabbing "EXITS" difference counts
    
    # Computes the "ENTRIES" + "EXITS" for each counter
    turnstile_df_valuecounts["TOTAL"] = turnstile_df_valuecounts["ENTRIES DIFFERENCE"] + turnstile_df_valuecounts["EXITS DIFFERENCE"]  
    
    # Grab names of "STATION" and number of times they appear in turnstile_df_valuecounts
    station_count = Counter(turnstile_df_valuecounts['STATION'])
    station_dict = {i:station_count[i] for i in station_count}
    
    # For each "STATION" name, sum up all the counts from all units
    value_count_dict = dict(zip(list(station_dict.keys()), [[turnstile_df_valuecounts.loc[turnstile_df_valuecounts['STATION'] == i, 'TOTAL'].sum()] for i in list(station_dict.keys())]))
    
    # Grab list of most popular stations and puts it into a sorted dataframe column
    value_count_df = pd.DataFrame.from_dict(value_count_dict).transpose() # Grabbing values
    value_count_df.columns = ["ValueCount"] # Renaming column to ValueCount
    value_count_df = value_count_df.sort_values(by=["ValueCount"], ascending=False) # Sorting by count

In [3]:
turnstile_df = pd.read_csv(r"Turnstile Data/Downloads/turnstile_190330.txt")
format_turnstile_df(turnstile_df)
turnstile_df # Preview

,CA,UNIT,SCP,STATION,LINENAME,DIVISION,TIME,DESC,ENTRIES,EXITS,DATETIMERAW
0,A002,R051,02-00-00,59 ST,NQR456W,BMT,1900-01-01 00:00:00,REGULAR,6989774,2370411,2019-03-23 00:00:00
1,A002,R051,02-00-00,59 ST,NQR456W,BMT,1900-01-01 04:00:00,REGULAR,6989795,2370413,2019-03-23 04:00:00
2,A002,R051,02-00-00,59 ST,NQR456W,BMT,1900-01-01 08:00:00,REGULAR,6989813,2370436,2019-03-23 08:00:00
3,A002,R051,02-00-00,59 ST,NQR456W,BMT,1900-01-01 12:00:00,REGULAR,6989924,2370512,2019-03-23 12:00:00
4,A002,R051,02-00-00,59 ST,NQR456W,BMT,1900-01-01 16:00:00,REGULAR,6990200,2370573,2019-03-23 16:00:00
5,A002,R051,02-00-00,59 ST,NQR456W,BMT,1900-01-01 20:00:00,REGULAR,6990562,2370623,2019-03-23 20:00:00
6,A002,R051,02-00-00,59 ST,NQR456W,BMT,1900-01-01 00:00:00,REGULAR,6990734,2370648,2019-03-24 00:00:00
7,A002,R051,02-00-00,59 ST,NQR456W,BMT,1900-01-01 04:00:00,REGULAR,6990758,2370653,2019-03-24 04:00:00
8,A002,R051,02-00-00,59 ST,NQR456W,BMT,1900-01-01 08:00:00,REGULAR,6990772,2370676,2019-03-24 08:00:00
9,A002,R051,02-00-00,59 ST,NQR456W,BMT,1900-01-01 12:00:00,REGULAR,6990860,2370731,2019-03-24 12:00:00


In [4]:
# Grab data between 8am and 12pm for the whole week
valuecount(dt.datetime(2019, 3, 23, 8, 0), dt.datetime(2019, 3, 23, 12, 0))
value_count_df_08to12 = value_count_df[["ValueCount"]]
value_count_df_08to12.columns = ["23"]

for i in range(24, 30):
    valuecount(dt.datetime(2019, 3, i, 8, 0), dt.datetime(2019, 3, i, 12, 0))
    value_count_df_08to12["{}".format(i)] = value_count_df[["ValueCount"]]

value_count_df_08to12.to_csv(r"ValueCount Data/valuecount_190330_df_08to12.csv")
value_count_df_08to12 # Preview

,23,24,25,26,27,28,29
34 ST-HERALD SQ,25176.0,21563.0,63791.0,66774.0,66130.0,67708.0,60023.0
34 ST-PENN STA,20202.0,19107.0,62738.0,65230.0,64952.0,65617.0,59090.0
TIMES SQ-42 ST,20062.0,14902.0,47673.0,50636.0,51769.0,50793.0,47351.0
FLUSHING-MAIN,15778.0,13458.0,25622.0,24946.0,25286.0,25467.0,24612.0
59 ST,14402.0,10672.0,32089.0,33663.0,34159.0,34388.0,32675.0
42 ST-PORT AUTH,14115.0,11135.0,35493.0,34578.0,37329.0,35119.0,31509.0
59 ST COLUMBUS,13942.0,11529.0,33727.0,35259.0,33759.0,36013.0,31776.0
86 ST,13641.0,10922.0,29429.0,30591.0,31581.0,31770.0,29980.0
GRD CNTRL-42 ST,13410.0,10688.0,43376.0,45776.0,44607.0,46089.0,41085.0
JKSN HT-ROOSVLT,13341.0,10460.0,20320.0,20683.0,20735.0,20985.0,20335.0


In [5]:
# Grab data between 12pm and 4pm for the whole week
valuecount(dt.datetime(2019, 3, 23, 12, 0), dt.datetime(2019, 3, 23, 16, 0))
value_count_df_12to16 = value_count_df[["ValueCount"]]
value_count_df_12to16.columns = ["23"]

for i in range(24, 30):
    valuecount(dt.datetime(2019, 3, i, 12, 0), dt.datetime(2019, 3, i, 16, 0))
    value_count_df_12to16["{}".format(i)] = value_count_df[["ValueCount"]]

value_count_df_12to16.to_csv(r"ValueCount Data/valuecount_190330_df_12to16.csv")
value_count_df_12to16 # Preview

,23,24,25,26,27,28,29
34 ST-HERALD SQ,40700.0,38454.0,43867.0,46261.0,47879.0,47244.0,51148.0
TIMES SQ-42 ST,34951.0,24895.0,31273.0,33252.0,36902.0,34758.0,37970.0
34 ST-PENN STA,31296.0,31864.0,38424.0,39698.0,40470.0,41576.0,46083.0
86 ST,23054.0,19206.0,24510.0,25485.0,25737.0,26855.0,28292.0
59 ST COLUMBUS,22925.0,21428.0,28915.0,30193.0,30406.0,31938.0,31385.0
59 ST,20674.0,16936.0,23107.0,24393.0,25416.0,25503.0,26778.0
GRD CNTRL-42 ST,20560.0,16642.0,24346.0,26003.0,26922.0,28212.0,29180.0
FLUSHING-MAIN,18083.0,17176.0,19607.0,19299.0,20467.0,19465.0,19550.0
CHAMBERS ST,17457.0,12978.0,25413.0,27054.0,27264.0,27046.0,28084.0
ATL AV-BARCLAY,16842.0,14482.0,15835.0,16250.0,16468.0,16782.0,17823.0


In [6]:
# Grab data between 4pm and 8pm for the whole week
valuecount(dt.datetime(2019, 3, 23, 16, 0), dt.datetime(2019, 3, 23, 20, 0))
value_count_df_16to20 = value_count_df[["ValueCount"]]
value_count_df_16to20.columns = ["23"]

for i in range(24, 30):
    valuecount(dt.datetime(2019, 3, i, 16, 0), dt.datetime(2019, 3, i, 20, 0))
    value_count_df_16to20["{}".format(i)] = value_count_df[["ValueCount"]]

value_count_df_16to20.to_csv(r"ValueCount Data/valuecount_190330_df_16to20.csv")
value_count_df_16to20 # Preview

,23,24,25,26,27,28,29
34 ST-HERALD SQ,47905.0,39556.0,86615.0,88372.0,89255.0,90113.0,85328.0
TIMES SQ-42 ST,36562.0,28193.0,62591.0,67917.0,69396.0,68892.0,66038.0
34 ST-PENN STA,33898.0,31061.0,78157.0,78323.0,79422.0,83548.0,78473.0
59 ST COLUMBUS,24166.0,20465.0,43885.0,46426.0,45504.0,46360.0,40377.0
86 ST,22782.0,16680.0,36351.0,38134.0,36717.0,38124.0,35400.0
GRD CNTRL-42 ST,20555.0,16257.0,50547.0,52165.0,52425.0,52877.0,48780.0
59 ST,20197.0,15764.0,38156.0,40146.0,40298.0,40926.0,37696.0
FLUSHING-MAIN,19467.0,16540.0,30309.0,31480.0,30759.0,31613.0,31370.0
ATL AV-BARCLAY,17247.0,13758.0,27344.0,28975.0,27533.0,28417.0,31694.0
23 ST,17217.0,12449.0,45414.0,48219.0,48878.0,49522.0,44160.0
